In [96]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, split, explode

import matplotlib.pyplot as plt
%matplotlib inline



In [97]:
spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [91]:
# spark is an existing SparkSession
iv = spark.read.csv("hdfs://localhost:9000/user/lavish/data/investments.csv"
                , header='true'
                , inferSchema='true')

In [92]:
iv.createOrReplaceTempView("investments")

In [93]:
type(iv)

pyspark.sql.dataframe.DataFrame

In [90]:
iv.schema.names

['company_permalink',
 'company_name',
 'company_category_list',
 'company_country_code',
 'company_state_code',
 'company_region',
 'company_city',
 'investor_permalink',
 'investor_name',
 'investor_country_code',
 'investor_state_code',
 'investor_region',
 'investor_city',
 'funding_round_permalink',
 'funding_round_type',
 'funding_round_code',
 'funded_at',
 'raised_amount_usd']

In [81]:
filteredIV = iv.select('raised_amount_usd','funded_at',split(col("company_category_list"), "[|]s*").alias("categoryArr")).filter(ShrinkedIV.raised_amount_usd.isNotNull())


In [103]:
filteredIV.show(50)

+-----------------+----------+--------------------+
|raised_amount_usd| funded_at|         categoryArr|
+-----------------+----------+--------------------+
|        2000000.0|19-03-2008|       [Curated Web]|
|          41250.0|01-07-2014|             [Games]|
|            2.0E7|09-11-2015|         [Analytics]|
|        3000000.0|22-05-2013|         [Analytics]|
|            2.0E7|09-11-2015|         [Analytics]|
|        1700000.0|03-01-2013|         [Analytics]|
|        8900000.0|19-07-2014|         [Analytics]|
|            2.0E7|09-11-2015|         [Analytics]|
|            2.0E7|09-11-2015|         [Analytics]|
|        8900000.0|19-07-2014|         [Analytics]|
|        1700000.0|03-01-2013|         [Analytics]|
|        1700000.0|03-01-2013|         [Analytics]|
|        8900000.0|19-07-2014|         [Analytics]|
|        8900000.0|19-07-2014|         [Analytics]|
|        8900000.0|19-07-2014|         [Analytics]|
|        8900000.0|19-07-2014|         [Analytics]|
|         15

In [99]:
explodedIV=filteredIV.select('raised_amount_usd','funded_at', explode('categoryArr'))

In [101]:
explodedIV.show(50)

+-----------------+----------+------------+
|raised_amount_usd| funded_at|         col|
+-----------------+----------+------------+
|        2000000.0|19-03-2008| Curated Web|
|          41250.0|01-07-2014|       Games|
|            2.0E7|09-11-2015|   Analytics|
|        3000000.0|22-05-2013|   Analytics|
|            2.0E7|09-11-2015|   Analytics|
|        1700000.0|03-01-2013|   Analytics|
|        8900000.0|19-07-2014|   Analytics|
|            2.0E7|09-11-2015|   Analytics|
|            2.0E7|09-11-2015|   Analytics|
|        8900000.0|19-07-2014|   Analytics|
|        1700000.0|03-01-2013|   Analytics|
|        1700000.0|03-01-2013|   Analytics|
|        8900000.0|19-07-2014|   Analytics|
|        8900000.0|19-07-2014|   Analytics|
|        8900000.0|19-07-2014|   Analytics|
|        8900000.0|19-07-2014|   Analytics|
|         150000.0|05-02-2014|      Mobile|
|        1000050.0|20-07-2011|      Mobile|
|        1000050.0|20-07-2011|      Mobile|
|         150000.0|05-02-2014|  

In [73]:
I = filteredIV.select(col('company_category_list'), split(col("company_category_list"), "[|]s*").alias("ev"))


In [75]:
I.show(50)

+---------------------+--------------------+
|company_category_list|                  ev|
+---------------------+--------------------+
|          Curated Web|       [Curated Web]|
|                Games|             [Games]|
|            Analytics|         [Analytics]|
|            Analytics|         [Analytics]|
|            Analytics|         [Analytics]|
|            Analytics|         [Analytics]|
|            Analytics|         [Analytics]|
|            Analytics|         [Analytics]|
|            Analytics|         [Analytics]|
|            Analytics|         [Analytics]|
|            Analytics|         [Analytics]|
|            Analytics|         [Analytics]|
|            Analytics|         [Analytics]|
|            Analytics|         [Analytics]|
|            Analytics|         [Analytics]|
|            Analytics|         [Analytics]|
|               Mobile|            [Mobile]|
|               Mobile|            [Mobile]|
|               Mobile|            [Mobile]|
|         

In [72]:
ef.show()

+---+-----------------+
| id|               ev|
+---+-----------------+
|  1|  [200# 201# 202]|
|  1|[lav, er, 34, 45]|
|  1|      [swer,sfsd]|
|  2|             [32]|
|  2|             null|
+---+-----------------+

